# 03 Precept: Least Squares

Agenda: 
- Lecture Review
- Coding Exercise

## Lecture Review - Least Squares



## Topics covered

 - Cholesky factorization

### Least Squares
- Least squares optimization
*  Gram matrix
-  Solving least squares



### Complexity of operations:

In the table below, $A$ and $B$ are $m \times n$ matrices, $C$ is an $m \times p$ matrix, $x$ is an $n$ -vector, and $a$ is a scalar.

Matrix operations  | Complexity (flops)
-------------------|------------------
$aA $     | $mn $
$A+B $|$mn $
$Ax$ | $2mn $
 $AC$ | $2mnp $
 $A^TA$| $mn^2$



### Factorization
 Matrix operations      | Complexity (flops)
------------------------|---------------------------------------------------
 $LU$ - Factorization   | $(2/3)n^3+2 n^{2} \approx (2 / 3) n^{3}$
 $LL^T$ - Factorization | $(1 / 3) n^{3}+2 n^{2} \approx (1 / 3) n^{3}$

### Solving normal equations of least squares problems:

Methods                                | Complexity (flops)
---------------------------------------|------------------
Inversion                              |  $O(n^3)$
Solving normal equations with Cholesky |$2 m n^{2}+2 m n+(1 / 3) n^{3}+2 n^{2} \approx 2 m n^{2}$


### Gram matrix and its properties 
  - Invertibility 
  - Positive (semi)definiteness

#### Clarification

- Positive (semi)definiteness v.s. Matrix comparison relationships 
    
    - Definition: Positive (semi)definiteness
    
    - On the other hand, for any two matrices  $M, Q \in \mathbb{R}^{N \times N}$ we write $M \gg Q$ if $m_{j k}>q_{j k}$, $M \succ Q$ if $m_{j k} \geq q_{j k},$ for any $j, k$, but $M \neq Q$, and $M \succeq Q$ if $m_{j k} \geq q_{j k}$ for any $j, k$.









# Sparsity of factorization vs inverse matrix

In [14]:
import numpy as np
from numpy.linalg import inv
from scipy.linalg import lu

n = 5
A = np.eye(n)
A[:, -1] = np.ones(n)
A[-1, :] = np.ones(n)
A

array([[1., 0., 0., 0., 1.],
       [0., 1., 0., 0., 1.],
       [0., 0., 1., 0., 1.],
       [0., 0., 0., 1., 1.],
       [1., 1., 1., 1., 1.]])

In [15]:
P, L, U = lu(A)
print("L = \n", L)
print("U = \n", U)

L = 
 [[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [1. 1. 1. 1. 1.]]
U = 
 [[ 1.  0.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  0.  1.  0.  1.]
 [ 0.  0.  0.  1.  1.]
 [ 0.  0.  0.  0. -3.]]


In [18]:
A_inv = inv(A)
print("A_inv = \n", A_inv)

A_inv = 
 [[ 0.66666667 -0.33333333 -0.33333333 -0.33333333  0.33333333]
 [-0.33333333  0.66666667 -0.33333333 -0.33333333  0.33333333]
 [-0.33333333 -0.33333333  0.66666667 -0.33333333  0.33333333]
 [-0.33333333 -0.33333333 -0.33333333  0.66666667  0.33333333]
 [ 0.33333333  0.33333333  0.33333333  0.33333333 -0.33333333]]


## Least squares orthogonality principle

The point $A x^\star$ is the linear combination of the columns of $A$ that is closest to $b .$ The optimal residual is $r=A x^\star -b .$ The optimal residual satisfies a property that is sometimes called the orthogonality principle:
It is orthogonal to the columns of $A,$ and therefore, it is orthogonal to any linear combination of the columns of $A$. In other words, for any $n$-vector $z$, we have
$$
(A z) \perp r \quad \iff \quad (Ax)^Tr = 0
$$
We can derive the orthogonality principle from the normal equations, which can be expressed as $A^{T}(A x^\star -b)=0$. For any $n$-vector $z$ we have
$$
(A z)^{T} r=(A z)^{T}(A x^\star -b)=z^{T} A^{T}(A x^\star - b) = 0
$$

![orthogonality](orthogonality.png)

The orthogonality principle is illustrated in the figure above, for a least squares problem with $m=3$ and $n=2$. The shaded plane is the set of all linear combinations $z_{1} a_{1}+z_{2} a_{2}$ of $a_{1}$ and $a_{2}$, the two columns of $A$. The point $A x^\star$ is the closest point in the plane to $b$. The optimal residual $r$ is shown as the vector from $b$ to $A x^\star$. This vector is orthogonal to any point in the shaded plane.

## Solving least-squares problems in Python

In [1]:
# Import required packages for coding exercises
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy.linalg import cholesky as llt
seed = 1
np.random.seed(seed) # We use np.random.seed for reproducible results

For convenience we have included the following function `lstsq` that solves the least squares problem as we have seen in class.

In [3]:
def forward_substitution(L, b):
    n = L.shape[0]
    x = np.zeros(n)
    for i in range(n):
        x[i] = (b[i] - L[i,:i] @ x[:i])/L[i, i]
    return x

def backward_substitution(U, b):
    n = U.shape[0]
    x = np.zeros(n)
    for i in reversed(range(n)):
        x[i] = (b[i] - U[i,i+1:] @ x[i+1:])/U[i, i]
    return x

def lstsq(A, b):
    M = A.T.dot(A)  # Form Gram matrix
    q = A.T.dot(b)  # Form right hand side
    L = llt(M)      # Factor
    x = forward_substitution(L, q)
    x = backward_substitution(L.T, x)
    return x, L

Generate a random $20\times 10$ matrix $A$ and a random $20$-vector $b$.

In [5]:
A = np.random.rand(20, 10)
b = np.random.rand(20, 1)

(a) Compare the solution $x^{\star}$ of the associated least-squares problem using pseudoinverse `np.linalg.pinv` and the method in class `lstsq` from above. Verify that the solutions are the same, or more accurately, very close to each other. They might be slightly different due to small roundoff errors.

(b) Let $x^{\star}$ be one of the solutions found in part (a). Now generate a random nonzero
$10$-vector $\delta$ and show that $\|A(x^{\star} +\delta)-b\|_2^2> \|Ax^{\star} -b\|_2^2$. Repeat several times with different values of $\delta$, you might try choosing a small $\theta$ by scaling the random vector obtained with `np.random.randn`